# Youtube Comments Extraction using Youtube API - Python

In [ ]:
### API configuration and creating youtube resource object
api_key = ' ' #fill out the string with your own api_key

from googleapiclient.discovery import build 
yt = build('youtube','v3',developerKey=api_key) #check out https://developers.google.com/docs/api/quickstart/python for more details

### Extracting the data
import pandas as pd

ID = 'eH1fFdjzJAw' #get the id from the youtube video link
box = [['Name', 'Comment', 'Time', 'Likes', 'Reply Count']] #we'd like to store the data in a list

def scrape_comments_with_replies():
    data = yt.commentThreads().list(part='snippet', videoId=ID, maxResults='100', textFormat="plainText").execute()
    #check out source:https://developers.google.com/youtube/v3/docs/commentThreads/list for more details

    for i in data["items"]:
        # Extracting the author name
        name = i["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
        # Extracting the comments
        comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
        # Extracting the publication date
        published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
        # Extracting the total likes
        likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
        # Extracting the total replies
        replies = i["snippet"]['totalReplyCount']
        
        # Append the data into the list
        box.append([name, comment, published_at, likes, replies])

        totalReplyCount = i["snippet"]['totalReplyCount']
        
        # Retieve the comment replies
        if totalReplyCount > 0:

            parent = i["snippet"]['topLevelComment']["id"]

            data2 = youtube.comments().list(part='snippet', maxResults='100', parentId=parent,
                                            textFormat="plainText").execute()

            for i in data2["items"]:
                name = i["snippet"]["authorDisplayName"]
                comment = i["snippet"]["textDisplay"]
                published_at = i["snippet"]['publishedAt']
                likes = i["snippet"]['likeCount']
                replies = ""

                box.append([name, comment, published_at, likes, replies])
    
    # Handling the pagination
    # The query below retrieves the next 100 additional page of results(maxResults='100')
    while ("nextPageToken" in data):

        data = yt.commentThreads().list(part='snippet', videoId=ID, pageToken=data["nextPageToken"],
                                             maxResults='100', textFormat="plainText").execute()

        for i in data["items"]:
            name = i["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
            comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
            published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
            likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
            replies = i["snippet"]['totalReplyCount']

            box.append([name, comment, published_at, likes, replies])

            totalReplyCount = i["snippet"]['totalReplyCount']

            if totalReplyCount > 0:

                parent = i["snippet"]['topLevelComment']["id"]

                data2 = youtube.comments().list(part='snippet', maxResults='100', parentId=parent,
                                                textFormat="plainText").execute()

                for i in data2["items"]:
                    name = i["snippet"]["authorDisplayName"]
                    comment = i["snippet"]["textDisplay"]
                    published_at = i["snippet"]['publishedAt']
                    likes = i["snippet"]['likeCount']
                    replies = ''

                    box.append([name, comment, published_at, likes, replies])

    # Save the retrieved data as a dataframe
    df = pd.DataFrame({'Name': [i[0] for i in box], 'Comment': [i[1] for i in box], 'Time': [i[2] for i in box],
                       'Likes': [i[3] for i in box], 'Reply Count': [i[4] for i in box]})

    df.to_csv('youtubecommentdata.csv', index=False, header=False)

    return "Data Retrieval is Successful!"

scrape_comments_with_replies()

In [ ]:
#quick check the data
data = pd.read_csv("youtubecommentdata.csv",sep=',')
data[1:10]
print(data['Comment'][9])